In [ ]:
# ==============================================================================
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [ ]:
!nvidia-smi

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: left;">

# DeepStream (Video Analytics Pipelines)

# Introduction

Deepstream applications introduce deep neural networks and other complex processing tasks into a stream processing pipeline to enable near real-time analytics on video and other sensor data. Extracting meaningful insights from these sensors creates opportunities for improved operational efficiences and safety. Cameras, for example, are the most deployed IoT sensor currently in use; cameras are found in our homes, on our streets, in parking los, shopping malls, warehouses, factories - they are everywhere. The potential use of video analytics is enormous: access control, loss prevention, automated checkout, surveillance, safety, automated inspection (QA), package sort (smart logistics), traffic control/engineering, industrial automation, etc.

<img src="images/ds_09.png" style="width: 1080px; float: center;">

Although intelligent video analytics (IVA) differs by industry and application, the flow from pixels to insights remains consistent across all use cases. It is this common workflow that is the foundation for the DeepStream SDK generic streaming analytics plug and play architecture.

If we think about a traditional computer vision pipeline, we will most likely like to do something like capture a set of video streams and then run some AI workload on each video stream.  To do that however, we also need to do things like decode each video stream and then pre-process or batch the videos/images so that the AI model knows how to process them.  After the AI models have run, we also may want to run some tracking if we are doing detection and then grab some metadata for further analytics.  Without a software stack, each of these components need to be developed manually and work in concert with each other flawlessly.

<img src="images/ds_04.png" style="width: 1080px; float: center;">

For that NVIDIA developed the DeepStream SDK which assists developers in creating entire pipelines for video analytics including the capture & decode, pre-processing, AI inference, tracking, analytics, and then composition for visualization.

<img src="images/ds_05.png" style="width: 1080px; float: center;">

One of the cool things about the DeepStream SDK which makes it useful across the board is it's ability to have each block or plugin run directly on a portion of the GPU.

For example, typically for decoding a video stream, there would be a need to develop a software decoder which would most likely not be optimized for performance (i.e. speed).  When using the DeepStream SDK, the configuration can utilize hardware acceleration with specialized units on the GPU itself for faster decoding.

Similar things can be said about each portion of the video analytics pipeline; the AI inference, tracker, and analytics software can be run directly on the GPU or even on the Deep Learning Accelerator (DLA) or the CPU.

<img src="images/ds_06.png" style="width: 1080px; float: center;">

When we think about a DeepStream application, we can think of each of the above operations as a block which is going to output some metadata or information to the next portion of the pipeline.  More specifically, a DeepStream application is a set of modular plugins connected to form a processing pipeline. Each plugin represents a functional block, e.g., inference using TensorRT, or multi-stream decode. Hardware accelerated plugins interact with the underlying hardware (where applicable) to deliver maximum performance. For example, the decode plugin interacts with NVDEC, and then inference plugin interacts with the GPU or DLA. Each plugin can be instantiated multiple times within a pipeline as needed.

## DeepStream SDK

The [NVIDIA DeepStream SDK](https://developer.nvidia.com/deepstream-sdk) is a streaming analytics toolkit based on the open source GStreamer multimedia framework. The DeepStream SDK accelerates development of scalable IVA applicatins, making it easier for developers to build core deep learning networks instead of designing end-to-end applications from scratch. The SDK is supported on systems that contain an NVIDIA Jetson module or an NVIDA dGPU adapter; it is comprisde of an extensible collection of hardware-accelerated plugins that interact with low-level libraries to optimize performance and defines a standardized metadata structure enabling custom/user-specific additions.

The materials and descriptions in this notebook provide a quick overview, for more detailed information and explanation of the DeepStream SDK refer to the following materials:

* [NVIDIA DeepStream SDK Development Guide](https://docs.nvidia.com/metropolis/deepstream/dev-guide/index.html)
* [NVIDIA DeepStream Plugin Manual](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/index.html)
* [NVIDIA DeepStream SDK API Reference Documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/DeepStream%20Development%20Guide/baggage/index.html)

### Reference Applications

The DeepStream SDK is packaged with several test applications including pre-trained models, example configuration files and sample video streams that can be used to run those applications. Additional samples and source code examples provide enough information to jumpstart development efforts for most IVA use cases. Test applications demonstrate:

* How to use DeepStream elements (e.g., get source, decode and mux multiple streams, run inference on pre-trained models, annotate and render image)
* How to generate a batch of frames and run inference on it for better resource utilization
* How to add custom/user-specific metadata to any component of DeepStream
* And much more... see the NVIDIA DeepStream SDK Development Guide for complete details

### GStreamer Plugins

GStreamer is a framework for plugins, data flow, and media type handling/negotiation. It is used to create streaming media applications. Plugins are shared libraries that are dynamically loaded at runtime and can be extended and upgraded independently. When arranged and linked together, plugins form the processing pipeline that defines the data flow for a streaming media application. You can learn more about GStreamer through its extensive online documentation, beginning with ["What is GStreamer?"](https://gstreamer.freedesktop.org/documentation/application-development/introduction/gstreamer.html?gi-language=c).

In addition to the open source plugins you'll find in the GStreamer framework libraries, the DeepStream SDK includes NVIDIA hardware accelerated plugins that leverage GPU capabilities. For a complete list of DeepStream GStreamer plugins, see the [NVIDIA DeepStream Plugin Manual](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/index.html#page/DeepStream_Plugin_Manual%2Fdeepstream_plugin_introduction.html).

Some of the noteable NVIDIA Hardware Accelerated plugins include:

- [Gst-nvstreammux](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/DeepStream_Plugin_Manual/deepstream_plugin_details.02.03.html) - Batch streams before sending for AI inference.
- [Gst-nvinfer](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/DeepStream_Plugin_Manual/deepstream_plugin_details.02.01.html#wwpID0E0IZ0HA) - Run inference using TensorRT.
- [Gst-nvvideo4linux2](https://docs.nvidia.com/metropolis/deepstream/4.0.1/plugin-manual/index.html#page/DeepStream_Plugin_Manual/deepstream_plugin_details.02.12.html) - Decode video streams using the hardware accelerated decoder (NVDEC); Encode RAW data in I420 format to H264 or H265 output video stream using hardware accelerated encoder (NVENC).
- [Gst-nvvideoconvert](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/DeepStream_Plugin_Manual/deepstream_plugin_details.02.07.html) - Perform video color format conversion. The first Gst-nvvideoconvert plugin before Gst-nvdsosd plugin converts stream data from I420 to RGBA and the second Gst-nvvideoconvert plugin after Gst-nvdsosd plugin converts data from RGBA to I420.
- [Gst-nvdsosd](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/DeepStream_Plugin_Manual/deepstream_plugin_details.02.06.html) - Draw bounding boxes, text, and region of interest (ROI) polygons.
- [Gst-nvtracker](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/DeepStream_Plugin_Manual/deepstream_plugin_details.02.02.html) - Track object between frames.
- [Gst-nvmultistreamtiler](https://docs.nvidia.com/metropolis/deepstream/plugin-manual/index.html#page/DeepStream_Plugin_Manual%2Fdeepstream_plugin_details.02.05.html) - Composite a 2D tile from batched buffers.
- [Gst-nvv4l2decoder](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v1.0/Docs/Accelerated_GStreamer_User_Guide.pdf) - Decode a video stream.
- [Gst-Nvv4l2h264enc](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v1.0/Docs/Accelerated_GStreamer_User_Guide.pdf) - Encode a video stream.
- [Gst-NvArgusCameraSrc](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v1.0/Docs/Accelerated_GStreamer_User_Guide.pdf) - Provide options to control ISP properties using the Argus API.

## GStreamer (Gst)

We won't go too in-depth into GStreamer in this particular lesson, but suffice it to say that DeepStream is built on top of GStreamer which is an open-source multimedia framework for building and constructing graphs of media-handling components (like video or audio).

Below you can try out the `gst-launch-1.0` command with a simple help option just to see the available options to you.

In [ ]:
!gst-launch-1.0 --help-all

We won't be running this as part of this lesson, but to build a simple pipeline that takes in a video (`input.h264`) and then runs some sort of inference on it and then returns the output video (`output.mp4`) could be written as something like this.

```bash
gst-launch-1.0 filesrc location=input.h264 \
    ! h264parse \
    ! nvv4l2decoder \
    ! m.sink_0 nvstreammux name=m batch-size=1 width=1920 height=1080 \
    ! nvinfer config-file-path=deepstream-config.txt \
    ! nvvideoconvert \
    ! nvdsosd \
    ! nvvideoconvert \
    ! ‘video/x-raw,format=I420’ \
    ! avenc_mpeg4 bitrate=2000000 \
    ! mpeg4videoparse \
    ! mux.video_0 qtmux name=mux \
    ! filesink location=output.mp4
```

As you can see, this can get very complicated very quickly especially as the pipeline gets more and more complicated.  As more codecs and more components become involved, it's much easier to make a mistake and debugging GStreamer pipelines that are written like the previous example is sometimes a pain.

For that reason, NVIDIA has created the DeepStream SDK which basically gives you the ability to create the same type of pipelines in a much easier manner.

---

# (1) Building a Pipeline with DeepStream Configuration Files (deepstream-app)

One of the ways to interact with DeepStream is to create/edit a configuration tile which tells the SDK how to construct the pipeline.

The Deepstream SDK comes with multiple sample configuration files that can be used out of the box for different use cases. Some of these include (these are located in the `/opt/nvidia/deepstream/deepstream-6.1/samples/configs/deepstream-app/` directory on your device where DeepStream was installed:

* **config_infer_primary.txt**: Configures a nvinfer element as primary detector.
* **config_infer_secondary_carcolor.txt**, **config_infer_secondary_carmake.txt**, **config_infer_secondary_vehicletypes.txt**: Configure a nvinfer element as secondary classifier.
* **config_tracker_IOU.txt**: Configures a low-level IOU (Intersection over Union) tracker.
* **config_tracker_NvDCF_accuracy.yml**: Config file for NvDCF tracker for higher accuracy.
* **config_tracker_NvDCF_max_perf.yml**: Config file for NvDCF tracker for max perf mode.
* **config_tracker_NvDCF_perf.yml**: Config file for NvDCF tracker for perf mode.
* **config_tracker_DeepSORT.yml**: Config file for DeepSORT tracker.
* **source1_usb_dec_infer_resnet_int8.txt**: Demonstrates one USB camera as input.
* **source2_1080p_dec_infer-resnet_demux_int8.txt**: Demonstrates demux mode for two sources.
* **source4_1080p_dec_infer-resnet_tracker_sgie_tiled_display_int8.txt**: (4 Decode + Infer + SGIE + Tracker)
* **source30_1080p_dec_infer-resnet_tiled_display_int8.txt**: (30 Decode + Infer)
* **sources_30.csv**: CSV file for 30 sources required in source30_1080p_dec_infer-resnet_tiled_display_int8.yml.
* **sources_4.csv**: CSV file for 30 sources required in source4_1080p_dec_infer-resnet_tracker_sgie_tiled_display_int8.yml.

We've created a few specialty configuration files based on these default ones specifically for this workshop.  The first configuration file we want to look at is [source2_mp4.txt](configs/source2_mp4.txt).  Feel free to check out the entire configuration file, but we will investigate a few key pieces below.

First, let's investigate the `source` portion of the configuration file (in this case, denoted `source0`.

In [ ]:
# printing lines 28 --> 39 in the sourc2_mp4.txt file

!sed -n 28,35p configs/source2_mp4.txt

Notice with the `source` portion of the configuration file (basically the pipeline input), we can set parameters like the type of video source (cameras, URI, RTSP, etc.) as well as memory types to use during the pipeline and GPU_ID (in this workshop, we only have 1 GPU, but on other machines, you could potentially have more). For this example, notice that we are using MultiURI, but only providing 1 URI and then specifying num-sources=2. This will just replicate the same video twice (in essence, streaming 2 videos, they just have the same content).

Next if we look at the `sink` portion of the configuration file, we can see a few options for streaming the output (i.e. output file, RTSP, etc.).

In [ ]:
!sed -n 41,83p configs/source2_mp4.txt

Here we can see a few different sink types. First is the `EglSink` which will act as a on on screen display in most cases. `sink1` represents output of the pipeline being streamed to a file (in this case an .mp4 file) which is what we will be using for this example. Notice with this option you can specify which codec to use (h264 or h265) as well as wehter to use the hardware encoder as part of the device or a software encoder for the video stream. The last sink, `sink2`, represents output to an RTSP stream. Once running, this can be viewed with most video streaming players (like VLC). To access, you can simply navigate to `rtsp://<ip-address>:8554/ds-test` and the browser will ask to open VLC (or equivalent on your local machine).

For the purposes of this lab, we will be using the second option (output to a file, mp4 video) so that it is viewable in Jupyter notebooks as shown in `sink1`.

A few other configuration parameters that we could modify include `tiled-display` for creating the output view in tiled format, `osd` for on-screen display settings, and `streammux` which handles the multiplexing of the multiple streams.

In this configuration, we also have set paths and properties for the models we wish to use for inference.

In [ ]:
!sed -n 122,180p configs/source2_mp4.txt

Here we can see each of the properties designated with a `gie` (GPU Inference engine) represent one of the models that will be used in the inference pipeline. In this case, we will be running a primary object detection network (i.e. `primary-gie`) which can detect vehicles and people. We then pass that information on to a couple of different classification models (i.e. `secondary-gie[0,1,2]`) which are for classifying vehicle type, vehicle color, and vehicle make, respectively.

Notice there is a also a `tracker` here (which is not enabled) that could be used for giving each instance of vehicle or person in the video it's own unique ID.  The tracker is able to be configured in an effort to speed up the pipeline by doing things like inferencing on one frame of the video stream and then running the tracker on a few subsequent frames instead of running the full detection model.

Lastly, notice for each of the `gie` sections, we provide a `config-file`. Inside these config files, you can find all of the components which correspond to the model itself including model path, batch size, classes, custom parsers, etc.

The image below depicts a similar pipeline to the one that we're building here (without the tracker and the message-broker which we'll touch later in the lab).

<img src="images/ds_16.png" style="width: 1080px; float: center;">

## Let's Try It

Now that we understand a little bit about the DeepStream configuration files which will help build the pipeline, we can now see a few examples of what the output looks like. Note again that for this demonstration we will be creating an mp4 as output instead of on-screen display since we are inside of a Jupyter notebook, but the config files can be modified to use any of the `sink` options at a later time.

First, let's copy our configuration files to the location where Deepstream was installed so that it's easier for it to access.

In [ ]:
!cp configs/* /opt/nvidia/deepstream/deepstream-6.1/samples/configs/deepstream-app/

Now to run the DeepStream SDK we can simply run `deepstream-app` with the appropriate parameters.

In [ ]:
!deepstream-app --help-all

Since we have put most of our configuration inside the config file we created earlier, we can simply use those instead of the command line arguments to successfully run `deepstream-app`.

## 2 Streams

Let's start with the 2 stream configuration that we discussed above (again, notice in the [configuration file](configs/source2_mp4.txt) that we are running the primary detector to detect vehicles and pedestrians as well as 3 secondary classifiers for car color, car make, and car model).

You'll notice that when you first run the `deepstream-app` that there appears to be an error.  This can be overlooked since the error is simply pointing out that the TensorRT engine does not exist and that it needs to build it in order to continue.  Also, reading through some of the output will show you that INT8 is not supported by this platform (namely, Jetson Nano); so eventhough we set INT8 mode in our configuration file, the app will instead try to create the next-best thing which in this case is FP16.

In [ ]:
!unset DISPLAY && deepstream-app -c /opt/nvidia/deepstream/deepstream-6.1/samples/configs/deepstream-app/source2_mp4.txt

In [ ]:
from IPython.display import HTML

HTML("""
    <video width="1280" height="720" controls>
        <source src="out_2streams.mp4" type="video/mp4">
    </video>
""")

## Adding a tracker

Now let's experiment with adding a tracker. The below table shows a couple of the trackers that are part of the DeepStream SDK but you are also able to implement your own custom tracker. In order to visualize the tracker more easily, we set the number of input sources to 1.

<img src="images/ds_12.png" style="width: 1080px; float: center;">

In [ ]:
# Note we've set the number of sources to 1
!sed -n 28,39p configs/source1_mp4_tracker.txt

In [ ]:
# This is how we add the tracker to our existing config file
!sed -n 137,151p configs/source1_mp4_tracker.txt

In [ ]:
!unset DISPLAY && deepstream-app -c /opt/nvidia/deepstream/deepstream-6.1/samples/configs/deepstream-app/source1_mp4_tracker.txt

In [ ]:
from IPython.display import HTML

HTML("""
    <video width="1280" height="720" controls>
        <source src="out_1streams_tracker.mp4" type="video/mp4">
    </video>
""")

## 8 Streams

Now, let's try something a little bigger. Now let's try to remove the secondary classifiers but increase the number of streams we are processing at once. For this example, we will try 8 streams with the object detection model.


In [ ]:
!unset DISPLAY && deepstream-app -c /opt/nvidia/deepstream/deepstream-6.1/samples/configs/deepstream-app/source8_mp4.txt

Notice for this batch (images from that video stream), we are processing and running our detection model across all 8 videos streams at about 160 FPS.

In [ ]:
from IPython.display import HTML

HTML("""
    <video width="1280" height="720" controls>
        <source src="out_8streams.mp4" type="video/mp4">
    </video>
""")

---
# (2) Building a Pipeline with the DeepStream Python API

Now that we have gone through and created a few different pipelines with configuration files, let's look at the Python API for building a similar pipeline.

Notice that the python API sits on top of the python API for GStreamer so should have most of the same functionality as you would get with the C/C++ API which sits on top of GStreamer.

<img src="images/ds_11.png" style="width: 1080px; float: center;">

In [ ]:
import sys
sys.path.append('/opt/nvidia/deepstream/deepstream-6.1/sources/deepstream_python_apps/apps/common')

import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst

from is_aarch_64 import is_aarch64
from bus_call import bus_call

import pyds

# import helper function 
from helper import *

#### Declaring class label ids and output video stream

In [ ]:
PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3

OUTPUT_VIDEO_NAME = "out_2streams_python.mp4"

#### Copy configuration file to deepstream_python_apps directory

For this particular example, we'll be using [configs/dstest1_pgie_config.txt](configs/dstest1_pgie_config.txt).  Feel free check out the contents and try to match up what you see with the configuration file which we used in the first section.

The cell below also highlights a portion of the configuration file that is important (i.e. defining the deep learning model we'll use, etc.)

In [ ]:
!sed -n 55,76p configs/dstest1_pgie_config.txt

In [ ]:
!cp configs/dstest1_pgie_config.txt /opt/nvidia/deepstream/deepstream-6.1/sources/deepstream_python_apps/apps/deepstream-test1/dstest1_py.txt

#### Initializing GStreamer

In [ ]:
#GObject.threads_init()
Gst.init(None);

#### Create GStreamer Pipeline to add elements to

In [ ]:
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n ")
pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")

#### Creating a source element for reading from a file

In [ ]:
print("Creating Source \n ")
source = Gst.ElementFactory.make("filesrc", "file-source")
if not source:
    sys.stderr.write(" Unable to create Source \n")

##### Creating a h264 parser as the input file is an elementary h264 stream

In [ ]:
print("Creating H264Parser \n")
h264parser = Gst.ElementFactory.make("h264parse", "h264-parser")
if not h264parser:
    sys.stderr.write(" Unable to create h264 parser \n")

##### Using nvdec_h264 for accelerated decoding on GPU

In [ ]:
print("Creating Decoder \n")
decoder = Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")
if not decoder:
    sys.stderr.write(" Unable to create Nvv4l2 Decoder \n")

##### Creating a nvstreammux instance to form batches for one or more sources

In [ ]:
print("Creating Streaming Multiplexer")
streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
if not streammux:
    sys.stderr.write(" Unable to create NvStreamMux \n")

##### Setting up nvinfer to run inference on the decoders output

**Note:** Behavior of inference is set through the config file

In [ ]:
print("Creating Inference Plugin")
pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
if not pgie:
    sys.stderr.write(" Unable to create pgie \n")

##### Using a converter to convert from NV12 to RGBA as required by nvosd

In [ ]:
print("Creating Video Converter (from NV12 to RGBA)")
nvvidconv = Gst.ElementFactory.make("nvvideoconvert", "convertor")
if not nvvidconv:
    sys.stderr.write(" Unable to create nvvideoconvert \n")

##### Create OSD to draw on the converted RGBA buffer

In [ ]:
print("Creating On-screen Display")
nvosd = Gst.ElementFactory.make("nvdsosd", "onscreendisplay")
if not nvosd:
    sys.stderr.write(" Unable to create nvosd \n")

##### Create another converter for use with the output to mp4

In [ ]:
print("Creating Video Converter (from RGBA to I420)")
nvvidconv2 = Gst.ElementFactory.make("nvvideoconvert", "converter2")
if not nvvidconv2:
    sys.stderr.write(" Unable to create nvvideoconvert")

##### Create caps filter for display

In [ ]:
print("Creating capsfilter")
capsfilter = Gst.ElementFactory.make("capsfilter", "capsfilter")
if not capsfilter:
    sys.stderr.write(" Unable to create capsfilter")
    
caps = Gst.Caps.from_string("video/x-raw, format=I420")
capsfilter.set_property("caps", caps)

##### Create encoder

In [ ]:
print("Creating Encoder")
encoder = Gst.ElementFactory.make("avenc_mpeg4", "encoder")
if not encoder:
    sys.stderr.write(" Unable to create encoder")
    
encoder.set_property("bitrate", 2000000)

##### Create code parser for mp4

In [ ]:
print("Creating Code Parser")
codeparser = Gst.ElementFactory.make("mpeg4videoparse", "mpeg4-parser")
if not codeparser:
    sys.stderr.write(" Unable to create codeparser")

##### Create container for mp4

In [ ]:
print("Creating Container")
container = Gst.ElementFactory.make("qtmux", "qtmux")
if not container:
    sys.stderr.write(" Unable to create container")

##### Create filesink

In [ ]:
print("Creating Filesink")
sink = Gst.ElementFactory.make("filesink", "filesink")
if not sink:
    sys.stderr.write(" Unable to create sink")

sink.set_property("location", OUTPUT_VIDEO_NAME)
sink.set_property("sync", 1)
sink.set_property("async", 0)

#### Setting PGIE and Streammux properties

In [ ]:
print("Playing file /opt/nvidia/deepstream/deepstream-6.1/samples/streams/sample_720p.h264 ")
source.set_property('location', \
                    "/opt/nvidia/deepstream/deepstream-6.1/samples/streams/sample_720p.h264")
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', 1)
streammux.set_property('batched-push-timeout', 4000000)
pgie.set_property('config-file-path', \
                  "/opt/nvidia/deepstream/deepstream-6.1/sources/deepstream_python_apps/apps/deepstream-test1/dstest1_py.txt")

#### Construct pipeline

Now we want to take all of the elements that we defined previously and build the pipeline which will be performing out video decoding, object detection, encoding, and writing out to an .mp4 file.

In [ ]:
# Add all elements to the pipeline
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(nvvidconv)
pipeline.add(nvvidconv2)
pipeline.add(encoder)
pipeline.add(capsfilter)
pipeline.add(codeparser)
pipeline.add(container)
pipeline.add(nvosd)
pipeline.add(sink)

# Start constructing the pipeline
# Add the source and connect it to the h264 parser
source.link(h264parser)
# Connect the h264 parser output to the decoder
h264parser.link(decoder)

sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    sys.stderr.write("Unable to get the sink pad of streammux \n")

srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write("Unable to get source pad of decoder \n")

# Connect the sink pad of the streammux to the src pad of the decoder
srcpad.link(sinkpad)
# Connect the PGIE (for inference)
streammux.link(pgie)
# Connect the Video Converter (from NV12 to RGBA)
pgie.link(nvvidconv)
# Connect OSD (to visualize the output/bounding boxes/etc.)
nvvidconv.link(nvosd)
# Connect the Video Converter (from RGBA to I420)
nvosd.link(nvvidconv2)
# Connect the Capsfilter
nvvidconv2.link(capsfilter)
# Connect encoder for mpeg4
capsfilter.link(encoder)
# Connect the code parser for mpeg4
encoder.link(codeparser)

sinkpad1 = container.get_request_pad("video_0")
if not sinkpad1:
    sys.stderr.write("Unable to get the sink pad of qtmux \n")

srcpad1 = codeparser.get_static_pad("src")
if not srcpad1:
    sys.stderr.write("Unable to get mpeg4 parse src pad \n")

# Connect sink pad from the container (i.e. the video output) to the source pad of the codeparser
srcpad1.link(sinkpad1)
# Create end of pipeline
container.link(sink);

#### Create an event loop and feeding bus messages to it

In [ ]:
loop = GObject.MainLoop()
#loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop);

#### Adding a probe to get the meta data generated

In [ ]:
osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write("Unable to get sink pad of nvosd \n")
    
osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0);

#### Starting the pipeline

In [ ]:
print("Starting pipeline \n")
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass
#cleanup
pipeline.set_state(Gst.State.NULL)

#### Setting up inferencing interval:

    0: inference every frame
    1: inference every other frame (e.g. detect objects every other frame, use tracker to localize in-between)
    2: inference every third frame
   
If desired, go to the [config file](configs/dstest1_pgie_config.txt) and change the interval as required.

**NOTE:** We won't be covering this in this portion of the lab, but feel free to play around with it later.

------

## Conclusion

In this notebook, we have demonstrated the process of creating a DeepStream pipeline using both the configuration files as well as the Python API as well as demonstrated using a custom model with a custom video stream to do detection and traffic monitoring.

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: left;">